https://data-ogauthority.opendata.arcgis.com/search?tags=Zip  
https://data-ogauthority.opendata.arcgis.com/datasets/oga-offshore-zipped-shapefiles-wgs84  
https://dataogauthority.blob.core.windows.net/external/OpenDataZips/OGA_OFF_WGS84.zip  


#### map tiles

https://en.wikipedia.org/wiki/Vector_tiles  
https://openmaptiles.com/downloads/planet/  

In [1]:

import geopandas
import ipyleaflet
import json
import numpy as np
import pandas as pd

from dateutil import parser as duparser

In [2]:
oga_pl = pd.read_csv("DealPipelinesKis.csv")

In [3]:
pldf = geopandas.read_file("OGA_data/OGA_OFF_WGS84/OGA_Pipelines_WGS84.shp")

In [4]:
pldf.head()

,OBJECTID,LICGEOE_ID,BLOCK,REASON,MOD_NOTE,NOTE,Shape_STAr,Shape_STLe,ID,NAME,...,CURR_LIC_6,OGA_COP,PIPE_DTINO,PIPE_NAME,DIAMETER,UNITS,FLUID,EXPOSED,UNTRENCHED,geometry
0,1,None,None,None,None,None,None,60.575787,510001891,None,...,None,None,PL4227,PL4227,8.0,inch,WATER,None,None,"LINESTRING (0.71534 57.42966, 0.71522 57.42929..."
1,2,None,None,None,None,None,None,6237.518751,500004203,None,...,None,None,None,NELSON TO SOUTH SATELLITE (R5A),10.0,inch,OIL,None,None,"LINESTRING (1.14430 57.66238, 1.14507 57.66285..."
2,3,None,None,None,None,None,None,4231.588723,500004285,None,...,None,None,PL1288,DURWARD MANIFOLD TO 1 WELLHEADS WATER,8.0,inch,WATER,Y,Y,"LINESTRING (0.16028 57.46607, 0.16044 57.46657..."
3,4,None,None,None,None,None,None,188796.758224,500004544,None,...,None,None,PL7N,HFC TO ST. FERGUS NORTH,32.0,inch,GAS,None,None,"LINESTRING (-0.28818 58.82726, -0.28828 58.827..."
4,5,None,None,None,None,None,None,3828.946051,510000242,None,...,None,None,PLU2767,FALCON TO KESTREL CONTROL UMBILICAL,3.0,inch,HYDRAULIC,None,None,"LINESTRING (0.98928 61.33007, 0.98890 61.32997..."


In [5]:
PL1844 = pldf.loc[pldf['PIPE_DTINO'] == "PL1844"]
PL1844

,OBJECTID,LICGEOE_ID,BLOCK,REASON,MOD_NOTE,NOTE,Shape_STAr,Shape_STLe,ID,NAME,...,CURR_LIC_6,OGA_COP,PIPE_DTINO,PIPE_NAME,DIAMETER,UNITS,FLUID,EXPOSED,UNTRENCHED,geometry
585,586,None,None,None,None,None,None,105206.633615,500005907,None,...,None,None,PL1844,CLAIR TO SULLOM VOE,22.0,inch,OIL,None,None,"LINESTRING (-2.54273 60.69127, -2.37246 60.699..."


In [6]:
idx = 200
pline = pldf.iloc[idx:idx+2]
pline

,OBJECTID,LICGEOE_ID,BLOCK,REASON,MOD_NOTE,NOTE,Shape_STAr,Shape_STLe,ID,NAME,...,CURR_LIC_6,OGA_COP,PIPE_DTINO,PIPE_NAME,DIAMETER,UNITS,FLUID,EXPOSED,UNTRENCHED,geometry
200,201,None,None,None,None,None,None,78.550745,500005843,None,...,None,None,PL1208.1,NEVIS SDU TO NEVIS N6 0.5IN CHEM,0.5,inch,CHEMICAL,None,None,"LINESTRING (1.40491 59.53151, 1.40495 59.53178..."
201,202,None,None,None,None,None,None,4767.075454,500004795,None,...,None,None,PL1729,CAPTAIN PRODUCTION HG FROM UTM,16.0,inch,OIL,None,None,"LINESTRING (-1.77104 58.30601, -1.77082 58.305..."


In [7]:
#countries_gdf.to_file("countries.geojson", driver='GeoJSON')
PL1844["geometry"].to_file("PL1844.geojson", driver='GeoJSON')

In [8]:
PL1844_json = pldf.to_json() #PL1844.to_json()
data = json.loads( PL1844_json )
#dd

In [9]:
# remove empty properties data
for _f in data['features']:
    _meta = {k:v for k,v in _f['properties'].items() if v is not None} #_f['properties'].copy()
    _id = _f['properties']['ID']
    _ogapl = oga_pl.loc[oga_pl['PIPELINEID'] == _id]
    if not _ogapl.empty:
        dd = _ogapl.iloc[0].to_dict()
        dd = {k:v for k,v in dd.items() if pd.notna(v) and str(v).strip() != "" }
        _meta.update(dd)

    new_meta = {}
    if 'PIPELINE_NAME' in _meta:
        new_meta["name"] = _meta['PIPELINE_NAME']
    elif 'PIPE_NAME' in _meta:
        new_meta["name"] = _meta['PIPE_NAME']
    elif 'PIPELINE_STDNAME' in _meta:
        new_meta["name"] = _meta['PIPELINE_STDNAME']
        
    if 'OPERATOR' in _meta:
        new_meta["operator"] = _meta['OPERATOR']
    elif 'PIPE_NAME' in _meta:
        new_meta["operator"] = _meta['NAMING_COMPANY']  

    if 'PIPELINE_DTINO' in _meta:
        new_meta["ID"] = _meta['PIPELINE_DTINO']
    elif 'PIPE_DTINO' in _meta:
        new_meta["ID"] = _meta['PIPE_DTINO']

    if 'DESCRIPTION' in _meta:
        new_meta["desc"] = _meta['DESCRIPTION']
    elif 'DESC_' in _meta:
        new_meta["desc"] = _meta['DESC_']

    #if 'ID' in _meta:
    #    new_meta["OGA_db_id"] = _meta['ID']
    #elif 'PIPELINEID' in _meta:
    #    new_meta["OGA_db_id"] = _meta['PIPELINEID']
    #elif 'EntityKey' in _meta:
    #    new_meta["OGA_db_id"] = _meta['EntityKey']

    if 'START_DATE' in _meta:
        new_meta["startup"] = duparser.parse(_meta['START_DATE']).date().isoformat()

    if 'PIPELINE_LENGTH_M' in _meta:
        length = float(_meta['PIPELINE_LENGTH_M'])
        if length<1000:
            new_meta["length"] = "{:d} m".format(int(length))
        else:
            new_meta["length"] = "{:.3g} km".format(length/1000.0) # int(_meta['PIPELINE_LENGTH_M'])

    if 'FLUID' in _meta:
        new_meta["fluid"] = _meta['FLUID']
    elif 'FLUID_CONVEYED' in _meta:
        new_meta["fluid"] = _meta['FLUID_CONVEYED']    

    if 'DIAMETER' in _meta:
        #new_meta["Dnom"] = _meta['DIAMETER']
        #new_meta["Do"] = _meta['DIAMETER']
        if ('UNITS' in _meta and _meta['UNITS'].lower()=='inch' or
           'DIAMETER_UNITS' in _meta and _meta['DIAMETER_UNITS'].lower()=='inch'):
            new_meta["Do"] = '{:.3g} in'.format(_meta['DIAMETER']) # _meta['DIAMETER']*25.4
        #new_meta["Do"] = int(new_meta["Do"]) # round(new_meta["Do"], 3)
        else:
            new_meta["Do"] = "{:d} mm".format(int(_meta['DIAMETER']))

    if 'STATUS' in _meta:
        new_meta["status"] = _meta['STATUS']

    #if 'COMMENTS' in _meta:
    #    new_meta["comments"] = _meta['COMMENTS']        

    #if 'INST_TYPE' in _meta:
    #    new_meta["INST_TYPE"] = _meta['INST_TYPE']         

    if 'INSULATION_COATING_TYPE' in _meta:
        new_meta["INSULATION_COATING_TYPE"] = _meta['INSULATION_COATING_TYPE']           

    if 'INTERNAL_DIAMETER_MM' in _meta:
        new_meta["Di"] = '{:.1g} mm'.format(_meta['INTERNAL_DIAMETER_MM']) # int(_meta['INTERNAL_DIAMETER_MM'])        

    if 'WALL_THICKNESS_MM' in _meta:
        new_meta["WT"] = int(_meta['WALL_THICKNESS_MM'])           
        
    if 'OPERATING_PRESSURE_MAX_BARG' in _meta:
        new_meta["MOP"] = "{:.1g} bar".format(_meta['OPERATING_PRESSURE_MAX_BARG']) # int(_meta['OPERATING_PRESSURE_MAX_BARG'])        
        
    _f['properties'] = new_meta.copy()
        #_f['properties'] = {k:v for k,v in _meta.items() if v is not None}
_meta #_f['properties']

{'DESC_': 'OSPREY TOWHEAD TO WELL P1 2IN ELECTRICAL UMBILICAL',
 'DIAMETER': 2.0,
 'END_DATE': '21-Sep-2019',
 'ID': 500004685,
 'INS_DATE': '20010920',
 'OBJECTID': 3541,
 'OPERATOR': 'FAIRFIELD',
 'PIPE_DTINO': 'PL754',
 'PIPE_NAME': 'OSPREY TOWHEAD TO P1',
 'STATUS': 'REMOVED',
 'Shape_STLe': 45.18930589602788,
 'UNITS': 'inch',
 'UPD_DATE': '20191106124144',
 'PIPELINE_DTINO': 'PL754',
 'PIPELINE_NAME': 'OSPREY TOWHEAD TO P1',
 'INST_TYPE': 'PIPELINE',
 'DESCRIPTION': 'OSPREY TOWHEAD TO WELL P1 2IN ELECTRICAL UMBILICAL',
 'COMMENTS': 'DECOMMISSIONED UMBILICAL, OSPREY PRODUCTION JUNCTION BOX TO OSPREY P1 ',
 'DIAMETER_UNITS': 'inch',
 'PIPELINE_LENGTH_M': 52.215740616650706,
 'ISSUED_DATE': '06-Sep-2001',
 'DATA_SOURCE': 'SHELL UK',
 'KP_ORIGIN_CRS': 23090.0,
 'KP_ORIGIN_CRS_DESC': 'ED_1950_TM_0_N',
 'NAMING_COMPANY': 'SHELL UK',
 'QC_FLAG': 'Y',
 'QC_ISSUE': 20.0,
 'QC_COMMENTS': 'DECOMMISSIONED REMOVED V39 -  ',
 'REASON_FOR_END': 'V39 FSIP submission -  ',
 'DATE_ENTERED': '20-Se

In [10]:
#pldf.to_file("OGA_pipelines.geojson", driver='GeoJSON')
with open("OGA_pipelines.geojson", 'w') as fh:
    json.dump(data, fh)

In [11]:
_ogapl.empty

False

In [12]:
pass

In [13]:
lmap = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.Esri.WorldImagery,
    center=(59.0, 2.0),
    zoom=6
)

lmap

Map(center=[59.0, 2.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [14]:
geo_json = ipyleaflet.GeoJSON(data=data, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
lmap.add_layer(geo_json)